In [ ]:
# import libraries
try:
  # Aggressively uninstall all tensorflow related packages
  !pip uninstall -y tensorflow tensorflow-datasets tf-keras tf-estimator-nightly
  # Reinstall stable TensorFlow and TensorFlow Datasets
  !pip install tensorflow tensorflow-datasets
except Exception as e:
  print(f"Error during TensorFlow installation: {e}")
  pass

import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

print(f"Training data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print("\nFirst few rows of training data:")
print(train_data.head())

In [ ]:
train_labels = train_data['label'].map({'ham': 0, 'spam': 1}).values
test_labels = test_data['label'].map({'ham': 0, 'spam': 1}).values

train_messages = train_data['message'].values
test_messages = test_data['message'].values

print(f"\nLabel distribution in training data:")
print(f"Ham: {np.sum(train_labels == 0)}, Spam: {np.sum(train_labels == 1)}")

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"


tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_messages)


train_sequences = tokenizer.texts_to_sequences(train_messages)
test_sequences = tokenizer.texts_to_sequences(test_messages)


train_padded = pad_sequences(train_sequences, maxlen=max_length,
                             padding=padding_type, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length,
                           padding=padding_type, truncating=trunc_type)

print(f"\nShape of padded training data: {train_padded.shape}")
print(f"Shape of padded test data: {test_padded.shape}")

In [ ]:
# prep the model

model = keras.Sequential([
    keras.layers.Embedding(vocab_size, 32, input_length=max_length),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

print(" model summary : \n")
model.summary()


In [ ]:
#train phase


num_epochs = 10
history = model.fit(
    train_padded,
    train_labels,
    epochs=num_epochs,
    validation_data=(test_padded, test_labels),
    verbose=2
)

In [ ]:
#Evaluate the Model

loss, accuracy = model.evaluate(test_padded, test_labels, verbose=2)
print(f"\nTest Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")



plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    sequence = tokenizer.texts_to_sequences([pred_text])

    padded_sequence = pad_sequences(sequence, maxlen=max_length,
                                    padding=padding_type, truncating=trunc_type)

    prediction = model.predict(padded_sequence, verbose=0)[0][0]


    label = 'spam' if prediction > 0.3 else 'ham'

    return [float(prediction), label]
pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
